In [26]:
import cv2
import numpy as np

# Capture video and initialize background subtractor
cap = cv2.VideoCapture('Images/traffic.mp4')
bg_subtractor = cv2.createBackgroundSubtractorKNN()

# Lucas-Kanade parameters
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read initial frame and set up points for tracking
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

kernel = np.ones((9, 9), dtype=np.uint8)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Background subtraction mask
    fg_mask = bg_subtractor.apply(gray)

    # Calculate optical flow for Lucas-Kanade    
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

    # Create a mask for optical flow points
    flow_mask = np.zeros_like(fg_mask)
    if p1 is not None:
        good_new = p1[st == 1]
        for point in good_new:
            x, y = point.ravel()
            flow_mask = cv2.circle(flow_mask, (int(x), int(y)), 5, 255, -1)  # Draw points as white on black mask

    # Merge masks
    combined_mask = cv2.bitwise_or(fg_mask, flow_mask)

    # Refine the mask (optional)
    kernel = np.ones((3, 3), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)

    # Remove very small bright areas
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)

    # Threshhold the mask so only the bright areas are detected
    _, combined_mask = cv2.threshold(combined_mask, 200, 255, cv2.THRESH_BINARY)

    # Make current white blobs bigger
    kernel = np.ones((9, 9), np.uint8)
    combined_mask = cv2.dilate(combined_mask, kernel, iterations=1)


    # Display the result
    cv2.imshow('Combined Mask', combined_mask)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Update previous frame and points
    old_gray = gray.copy()
    p0 = good_new.reshape(-1, 1, 2)


                                


cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [15]:
import cv2
import numpy as np
from scipy.optimize import linear_sum_assignment

# Video and background subtraction initialization
video = cv2.VideoCapture("Images/traffic.mp4")

bg_subtractor = cv2.createBackgroundSubtractorKNN()
bg_subtractor.setShadowThreshold(0.5)

# Lucas-Kanade parameters
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read initial frame and set up points for tracking
ret, old_frame = video.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, maxCorners=100, qualityLevel=0.01, minDistance=7, blockSize=7)


kernel = np.ones((9, 9), dtype=np.uint8)

# Kalman filter tracking structures
tracked_objects = {}
next_object_id = 0
max_missed_frames = 10  # Max frames to keep an object if it’s not detected
min_appearance_frames = 3  # Minimum frames for an ID to be shown

lower_shadow = np.array([0, 0, 50])
upper_shadow = np.array([180, 60, 200])

def create_kalman_filter():
    kf = cv2.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kf.processNoiseCov = np.eye(4, dtype=np.float32) * 0.01
    return kf

def initialize_kalman_for_object(center):
    global next_object_id
    kf = create_kalman_filter()
    kf.statePre = np.array([center[0, 0], center[1, 0], 0, 0], dtype=np.float32)
    kf.correct(center)
    tracked_objects[next_object_id] = {
        'kf': kf,
        'id': next_object_id,
        'missed_frames': 0,
        'appearance_count': 0  # Tracks how many consecutive frames the object has appeared
    }
    next_object_id += 1


def update_tracked_objects(detected_centers):
    global tracked_objects
    unmatched_detections = detected_centers[:]
    tracked_ids = list(tracked_objects.keys())
    predicted_positions = [tracked_objects[obj_id]['kf'].predict()[:2].flatten() for obj_id in tracked_ids]

    # Create cost matrix (distances between detected and predicted centers)
    cost_matrix = np.zeros((len(tracked_ids), len(detected_centers)), dtype=np.float32)
    for i, predicted in enumerate(predicted_positions):
        for j, detected in enumerate(detected_centers):
            cost_matrix[i, j] = np.linalg.norm(predicted - detected.flatten())

    # Solve assignment problem
    if len(tracked_ids) > 0 and len(detected_centers) > 0:
        row_ind, col_ind = linear_sum_assignment(cost_matrix)

        matched_detections = set()
        # Update matched tracked objects
        for row, col in zip(row_ind, col_ind):
            if cost_matrix[row, col] < 50:  # Threshold for matching
                obj_id = tracked_ids[row]
                tracked_objects[obj_id]['kf'].correct(detected_centers[col])
                tracked_objects[obj_id]['missed_frames'] = 0
                tracked_objects[obj_id]['appearance_count'] += 1
                matched_detections.add(col)
            else:
                # If the distance is too large, mark as unmatched
                tracked_objects[tracked_ids[row]]['missed_frames'] += 1
                tracked_objects[tracked_ids[row]]['appearance_count'] = 0

        unmatched_detections = [detected_centers[i] for i in range(len(detected_centers)) if i not in matched_detections]
    else:
        unmatched_detections = detected_centers

    # Increment missed frames for unmatched tracked objects
    for obj_id in tracked_ids:
        if obj_id not in [tracked_ids[row] for row in row_ind]:
            tracked_objects[obj_id]['missed_frames'] += 1
            tracked_objects[obj_id]['appearance_count'] = 0

        # Remove objects lost for too many frames
        if tracked_objects[obj_id]['missed_frames'] > max_missed_frames:
            del tracked_objects[obj_id]

    # Initialize new objects for unmatched detections
    for center in unmatched_detections:
        initialize_kalman_for_object(center)

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Background subtraction and thresholding
    fg_mask = bg_subtractor.apply(frame)
    # fg_mask[fg_mask < 200] = 0

    # Morphological operations for noise removal
    kernel = np.ones((4, 4), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow for Lucas-Kanade
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

    # Create a mask for optical flow points
    flow_mask = np.zeros_like(fg_mask)
    if p1 is not None:
        good_new = p1[st == 1]
        for point in good_new:
            x, y = point.ravel()
            flow_mask = cv2.circle(flow_mask, (int(x), int(y)), 5, 255, -1)  # Draw points as white on black mask

    # Merge masks
    combined_mask = cv2.bitwise_or(fg_mask, flow_mask)

    # Convert mask to HSV
    combined_mask_bgr = cv2.cvtColor(combined_mask, cv2.COLOR_GRAY2BGR)
    combined_mask_hsv = cv2.cvtColor(combined_mask_bgr, cv2.COLOR_BGR2HSV)
    
    shadow_mask = cv2.inRange(combined_mask_hsv, lower_shadow, upper_shadow)
    combined_mask = cv2.bitwise_and(combined_mask, combined_mask, mask=cv2.bitwise_not(shadow_mask))

    
    # Remove very small bright areas
    kernel = np.ones((4, 4), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)


    # Make current white blobs bigger
    kernel = np.ones((9, 9), np.uint8)
    combined_mask = cv2.dilate(combined_mask, kernel, iterations=1)


    # Detect contours
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_centers = [np.array([[x + w / 2], [y + h / 2]], dtype=np.float32) 
                        for contour in contours if cv2.contourArea(contour) > 750 
                        for x, y, w, h in [cv2.boundingRect(contour)]]
    
    # Draw bounding boxes
    for contour in contours:
        if cv2.contourArea(contour) > 750:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    


    # Update tracked objects based on detected centers
    update_tracked_objects(detected_centers)
    
    # Predict and draw tracked objects
    for obj_id, data in tracked_objects.items():
        kf = data['kf']
        predicted = kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        
        # Only display the ID if the appearance count exceeds the threshold
        if data['appearance_count'] >= min_appearance_frames:
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
            cv2.putText(frame, f'ID {data["id"]}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display frames
    cv2.imshow('Frame', frame)
    # cv2.imshow('Combined Mask', combined_mask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Update previous frame and points
    old_gray = gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

# Clean up
video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)



-1